# Central Controller FedADV Optimization

TJ Kim

12.24.21

#### Summary:
- Central controller solves for proportion of adv datapoint for each of the users FedEM Case
- Respects the resource constraints (proportion) of each of the nodes 
- Minimizes objective: Closest to proportion for each of the hypotheses scaled by datapoints

In [1]:
import numpy as np
import copy

In [2]:
# Argument inputs
G = 0.4 # total desired proportion
N = 30 # Number of users 
num_h = 3
Du = np.random.randint(low=50,high=400,size=N) # Dataset size of each user
D = np.sum(Du) # Total number of data points

# Hypothesis weights of each user
Whu = np.random.uniform(low=0.0, high=1, size=[N,num_h]) # Hypothesis weight for each user
row_sums = Whu.sum(axis=1)
Whu = Whu / row_sums[:, np.newaxis]
Wh = np.sum(Whu,axis=0)/N

S = 0.01 # Threshold
Ru = np.random.uniform(low=0.0, high=1, size=N) # Resource constraint per user
step_size = 0.01

In [3]:
# Solve for Fu for all users
def solve_proportions(G, N, num_h, Du, Whu, S, Ru, step_size):
    """
    Inputs:
    - G - Desired proportion of adv data points
    - N - Number of users in the system
    - num_h - Number of mixtures/hypotheses (FedEM)
    - Du - Number of data points at user U
    - Whu - Weight of each hypothis at user U
    - S - Threshold for objective function to fall below
    - Ru - Resource limits at each user (proportion)
    - step_size - For sweeping Fu
    Output:
    - Fu - proportion of adv data for each client
    """
    
    # finalize information needed to solve problem
    Wh = np.sum(Whu,axis=0)/N
    D = np.sum(Du)

    Fu = np.ones_like(Ru) * G

    # Step 1. Initial filter out all users with less resource constraints
    A = np.where(Fu>Ru)[0]
    B = np.where(Fu<Ru)[0]
    Fu[A] = Ru[A]

    # Step 2. Select users at random and change proportion, check objective 
    np.random.shuffle(B)
    for i in B:
        curr_obj = calc_prop_objective(G, num_h, Du, Whu, Fu)
        while Fu[i] + step_size < Ru[i]:
            Fu_temp = copy.deepcopy(Fu)
            Fu_temp[i] = Fu[i] + step_size
            new_obj = calc_prop_objective(G, num_h, Du, Whu, Fu_temp)
            if new_obj <= S:
                Fu = Fu_temp
                break
            elif new_obj < curr_obj:
                Fu = Fu_temp
                curr_obj = new_obj
            else: break
                
    return Fu

def calc_prop_objective(G, num_h, Du, Whu, Fu):
# Calculate objective function value for attaining global adv data proportion
    obj = 0
    D = np.sum(Du)
    for n in range(num_h):    
        obj += np.abs(np.sum(Fu * Du * Whu[:,n])- G * D * Wh[n]) * 1/D
    return obj

In [5]:
Fu = solve_proportions(G, N, num_h, Du, Whu, S, Ru, step_size)

In [10]:
Fu

array([0.41      , 0.55      , 0.5       , 0.41      , 0.1593135 ,
       0.41      , 0.4       , 0.1857959 , 0.41      , 0.87      ,
       0.39349449, 0.41      , 0.41      , 0.41      , 0.14608693,
       0.02603444, 0.41      , 0.66      , 0.14691094, 0.53      ,
       0.41      , 0.97      , 0.00537223, 0.1360616 , 0.41      ,
       0.41      , 0.41      , 0.41      , 0.13599737, 0.41      ])

In [8]:
calc_prop_objective(G, num_h, Du, Whu, Fu)

0.008779971628109739